In [1]:
"""
# 3 pytorch model(컨볼루션 레이어 2개 추가 & 에폭수 300)
깊은 신경망을 이용한 classification 문제 해결
"""

'\n# 2 pytorch model\n깊은 신경망을 이용한 classification 문제 해결\n'

In [2]:
"""
# 첫번째 레이어의 형태 (Convolutional Layer)
합성곱(in_channel=1, out_channel=32, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)
MaxPooling(kernel_size=2, stride=2)

# 두번째 레이어의 형태 (Convolutional Layer)
합성곱(in_channel=32, out_channel=64, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)
MaxPooling(kernel_size=2, stride=2)

# 세번째 레이어의 형태 (Convolutional Layer)
합성곱(in_channel=64, out_channel=128, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)
MaxPooling(kernel_size=2, stride=2, padding=1)

# 네번째 레이어의 형태 (Convolutional Layer)
합성곱(in_channel=128, out_channel=256, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)
MaxPooling(kernel_size=2, stride=2, padding=1)

# 다섯째 레이어의 형태 (Convolutional Layer)
합성곱(in_channel=256, out_channel=512, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)
MaxPooling(kernel_size=2, stride=2, padding=1)

# 여섯번째 레이어 : 전결합층(FC layer)
특성맵을 펼치는 역할
FC(Fully connected) layer + 활성화 함수(ReLU)

# 일곱번째 레이어: 전결합층(FC layer)
FC(Fully connected) layer + 활성화 함수(Softmax)
(마지막 노드는 10개) -> 0~9까지의 정답을 가져야 하기 때문에 노드를 10개를 설정
"""

'\n# 첫번째 레이어의 형태 (Convolutional Layer)\n합성곱(in_channel=1, out_channel=32, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)\nMaxPooling(kernel_size=2, stride=2)\n\n# 두번째 레이어의 형태 (Convolutional Layer)\n합성곱(in_channel=1, out_channel=64, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)\nMaxPooling(kernel_size=2, stride=2)\n\n# 세번째 레이어의 형태 (Convolutional Layer)\n합성곱(in_channel=1, out_channel=128, kernel_size=3, stride=1, padding=1) + 활성화 함수(ReLU)\nMaxPooling(kernel_size=2, stride=2, padding=1)\n\n# 네번째 레이어 : 전결합층(FC layer)\n특성맵을 펼치는 역할\nFC(Fully connected) layer + 활성화 함수(ReLU)\n\n# 다섯번째 레이어: 전결합층(FC layer)\nFC(Fully connected) layer + 활성화 함수(Softmax)\n(마지막 노드는 10개) -> 0~9까지의 정답을 가져야 하기 때문에 노드를 10개를 설정\n'

In [3]:
# 실습 진행에 필요한 라이브러리 import
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [4]:
# GPU 사용을 위한 설정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드를 고정
torch.manual_seed(777)

# 위의 고정 랜덤 시드는 GPU가 사용 가능 할 경우에 고정됨
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [5]:
# 학습에 필요한 파라미터 설정하기
learning_rate = 0.001
training_epochs = 300
batch_size = 100

In [6]:
# torchvision 데이터로드의 데이터셋을 정의
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [7]:
# 배치 크기를 지정하는 과정
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

# 60,000개의 데이터가 batch_size 100의 크기로 진행되기 때문에 total batch는 600이 됨

In [19]:
# 클래스를 이용하여 모델을 설계하기
class CNN(torch.nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.keep_prob = 0.5
        
        # 첫번째 레이어
        self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
                                          torch.nn.ReLU(),
                                          torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # 두번째 레이어
        self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # 세번째 레이어
        self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(64, 128,kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        # 네번째 레이어
        self.layer4 = torch.nn.Sequential(torch.nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        # 다섯번째 레이어
        self.layer5 = torch.nn.Sequential(torch.nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1),
                                         torch.nn.ReLU(),
                                         torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=1))
        # 여섯번째 레이어
        self.fc1 = torch.nn.Linear(2 * 2 * 512, 1024, bias=True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        self.layer6 = torch.nn.Sequential(self.fc1,
                                         torch.nn.ReLU(),
                                         torch.nn.Dropout(p=1 - self.keep_prob)) # 드롭아웃 : 과적합 방지
        
        # 다섯번째 레이어
        self.fc2 = torch.nn.Linear(1024, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc2.weight)
        
    # 순전파 전달을 위한 함수 forward 선언
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = out.view(out.size(0), -1) # 텐서를 펼치는 역할 : Flatten
        out = self.layer6(out)
        out = self.fc2(out)
        
        return out

In [20]:
# CNN 모델을 정의
model = CNN().to(device)

In [21]:
# 비용함수와 옵티마이저 선언
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
# 총 배치의 수 확인하기(파라미터 조정하기)
total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

# 사용자가 지정한 배치의 수에 따라 총 배치의 수가 달라짐을 볼 수 있음
# 배치 횟수 변경, 에폭 횟수 변경, 옵티마이저 변경, 손실함수 변경(비용함수) 등을
# 여러 방법으로 변경해보며 어떤 환경이 최적이었는지 판단하는 과정을 거쳐, 사용에 있어서 최적의 모델을 만들어 냄

총 배치의 수 : 600


In [23]:
# 학습 진행
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.222361535
[Epoch:    2] cost = 0.0522321537
[Epoch:    3] cost = 0.0411277153
[Epoch:    4] cost = 0.0302186571
[Epoch:    5] cost = 0.0270533171
[Epoch:    6] cost = 0.0231900513
[Epoch:    7] cost = 0.0183488876
[Epoch:    8] cost = 0.0188881159
[Epoch:    9] cost = 0.0151637727
[Epoch:   10] cost = 0.0136885857
[Epoch:   11] cost = 0.0137856854
[Epoch:   12] cost = 0.0113630304
[Epoch:   13] cost = 0.00976940338
[Epoch:   14] cost = 0.0139804808
[Epoch:   15] cost = 0.0108144097
[Epoch:   16] cost = 0.00786576699
[Epoch:   17] cost = 0.0108803026
[Epoch:   18] cost = 0.00816813763
[Epoch:   19] cost = 0.00856432877
[Epoch:   20] cost = 0.0102021331
[Epoch:   21] cost = 0.00829034112
[Epoch:   22] cost = 0.00574529264
[Epoch:   23] cost = 0.0108382236
[Epoch:   24] cost = 0.0103087481
[Epoch:   25] cost = 0.00574399065
[Epoch:   26] cost = 0.0108023379
[Epoch:   27] cost = 0.00654475205
[Epoch:   28] cost = 0.00641290843
[Epoch:   29] cost = 0.00532318279
[Epoc

[Epoch:  238] cost = 0.0065633026
[Epoch:  239] cost = 0.00214537652
[Epoch:  240] cost = 0.00876693707
[Epoch:  241] cost = 0.0119822752
[Epoch:  242] cost = 0.00644715223
[Epoch:  243] cost = 0.00873367861
[Epoch:  244] cost = 0.00894348975
[Epoch:  245] cost = 0.00515747536
[Epoch:  246] cost = 0.0211396851
[Epoch:  247] cost = 0.00526049547
[Epoch:  248] cost = 0.00135715632
[Epoch:  249] cost = 0.000703379803
[Epoch:  250] cost = 8.45880641e-05
[Epoch:  251] cost = 1.8050061e-05
[Epoch:  252] cost = 0.0259509757
[Epoch:  253] cost = 0.0101925032
[Epoch:  254] cost = 0.00311058899
[Epoch:  255] cost = 0.00393574266
[Epoch:  256] cost = 0.0064333654
[Epoch:  257] cost = 0.00966190081
[Epoch:  258] cost = 0.0114308754
[Epoch:  259] cost = 0.0221194476
[Epoch:  260] cost = 0.00389148295
[Epoch:  261] cost = 0.000333345204
[Epoch:  262] cost = 0.0104141133
[Epoch:  263] cost = 0.0119087687
[Epoch:  264] cost = 0.00917410292
[Epoch:  265] cost = 0.0194809828
[Epoch:  266] cost = 0.00634

In [24]:
# 모델 테스트
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    
    print("Accuracy : ", accuracy.item())

Accuracy :  0.9298999905586243


In [ ]:
# [결론]
# 에폭 수를 늘려서 학습을 많이한다고 해서 accuracy 높아지는 것 아님
# 더 깊은 레이어를 만든다고 해서 accuracy 높아지는 것 아님